In [17]:
from lusidtools.jupyter_tools import toggle_code

"""Get History of an Entity

This notebook shows how to get the full history of an entity using the entities asAt history.

Attributes
----------
entities
history
instruments
"""

toggle_code("Hide docstring")

# Get History of an Entity

In this notebook, we cover how to get the full history of an entity.

In this case, our entity is an `Instrument`, but  you can apply this technique to the following entities:
- `Abor`
- `Abor Config`
- `Allocation`
- `Block`
- `Corporate Action Source`
- `Custom Entity`
- `Execution`
- `Instrument`
- `Legal Entity`
- `Order`
- `Order Graph`
- `Order Instruction`
- `Package`
- `Participation`
- `Person`
- `Placement`
- `Portfolio`
- `Portfolio Group`
- `Relationship Definition`
- `Transaction Fee`
- `Transaction Portfolio`


To get the history of an entity, the steps are as follows:
1. [Create Entity](#1.-Create-Entity)
2. [Modify Entity](#2.-Modify-Entity)
3. [Get History of Entity](#3.-Get-History-of-Entity)

## Setup

Firstly, let's import LUSID packages

In [1]:
# Import generic python modules
import pandas as pd
from datetime import datetime, timedelta
import pytz
import os

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import format_quotes_response
from lusidjam import RefreshingToken

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path
)

Next, let's create our API.

In [2]:
# LUSID Variable Definitions
instruments_api = api_factory.build(lu.api.InstrumentsApi)

Finally, we declare some helper functions.

In [3]:
def create_instrument(name, client_internal, figi):
    instr_result = instruments_api.upsert_instruments(
        request_body={
            name: lm.InstrumentDefinition(
                name=name,
                identifiers={
                    "ClientInternal": lm.InstrumentIdValue(value=client_internal),
                    "Figi": lm.InstrumentIdValue(value=figi),
                }
            )
        }
    )
    # luid
    return instr_result.values[name].lusid_instrument_id

def update_instrument(luid, figi):
    update_result = instruments_api.update_instrument_identifier(
        identifier_type="LusidInstrumentId",
        identifier=luid,
        update_instrument_identifier_request=lm.UpdateInstrumentIdentifierRequest(
            type="Figi",
            value=figi
        )
    )
    # luid
    return update_result.lusid_instrument_id

def get_asat(luid, as_at=None):
    response = instruments_api.get_instrument(
        identifier_type="LusidInstrumentId",
        identifier=luid,
        as_at=as_at
    )
    return response

## 1. Create Entity

First create the entity. In this case we are creating an instrument.

In [4]:
luid = create_instrument(name="AsAtInstrument", client_internal=f"AS_AT_INSTRUMENT", figi="Original")
luid

'LUID_00003IZ7'

## 2. Modify Entity

Let's modify the entity. For our example, we will be changing the FIGI of the instrument.

In [5]:
update_instrument(luid=luid, figi="Edit1")

'LUID_00003IZ7'

In [8]:
update_instrument(luid=luid, figi="Edit2")

'LUID_00003IZ7'

In [9]:
update_instrument(luid=luid, figi="Original")

'LUID_00003IZ7'

## 3. Get History of Entity

In order to get the history of an entity, we need to use the version `asAt` data that is returned.

The data looks like the following:

In [15]:
resp = get_asat(luid).version
resp

{'as_at_created': datetime.datetime(2023, 5, 3, 13, 39, 16, 512886, tzinfo=tzlocal()),
 'as_at_date': datetime.datetime(2023, 5, 3, 13, 51, 14, 991585, tzinfo=tzlocal()),
 'as_at_modified': datetime.datetime(2023, 5, 3, 13, 40, 8, 387528, tzinfo=tzlocal()),
 'as_at_version_number': 4,
 'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'user_id_created': '00uieujafoYdmkDSx2p7',
 'user_id_modified': '00uieujafoYdmkDSx2p7'}

The key aspect is the `as_at_modified` value. If we fetch our entity from a date and time that is slightly before this value, we will see we have a lower `as_at_version_number`. See the following example and how the version number changes from `4` to `3`.

In [16]:
current_as_at = resp.as_at_modified
previous_as_at = current_as_at - timedelta(milliseconds=1)

get_asat(luid, previous_as_at).version

{'as_at_created': datetime.datetime(2023, 5, 3, 13, 39, 16, 512886, tzinfo=tzlocal()),
 'as_at_date': datetime.datetime(2023, 5, 3, 13, 40, 8, 386528, tzinfo=tzlocal()),
 'as_at_modified': datetime.datetime(2023, 5, 3, 13, 40, 7, 246434, tzinfo=tzlocal()),
 'as_at_version_number': 3,
 'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'user_id_created': '00uieujafoYdmkDSx2p7',
 'user_id_modified': '00uieujafoYdmkDSx2p7'}

We can do this recursively to get the **full history** of an entity.

In [12]:
as_at = pytz.utc.localize(datetime.now())
instr_data = {
    "date": [],
    "version_number": [],
    "modified_id": [],
    "client_internal": [],
    "figi": [],
    "luid": []
}

while True:
    resp = get_asat(luid, as_at)
    ids = resp.identifiers
    version = resp.version
    # add instr data to df
    instr_data['date'].append(version.as_at_modified)
    instr_data['version_number'].append(version.as_at_version_number)
    instr_data['modified_id'].append(version.user_id_modified)
    instr_data['client_internal'].append(ids['ClientInternal'])
    instr_data['figi'].append(ids['Figi'] if 'Figi' in ids else None)
    instr_data['luid'].append(ids['LusidInstrumentId'])
    # final version, end
    if version.as_at_version_number == 1:
        break
    # get previous version by going slightly before previous modified date
    as_at = version.as_at_modified - timedelta(milliseconds=1)

pd.DataFrame(data=instr_data).sort_values(by='date')

,date,version_number,modified_id,client_internal,figi,luid
3,2023-05-03 13:39:16.512886+00:00,1,00uieujafoYdmkDSx2p7,AS_AT_INSTRUMENT,Original,LUID_00003IZ7
2,2023-05-03 13:39:16.701563+00:00,2,00uieujafoYdmkDSx2p7,AS_AT_INSTRUMENT,Edit1,LUID_00003IZ7
1,2023-05-03 13:40:07.246434+00:00,3,00uieujafoYdmkDSx2p7,AS_AT_INSTRUMENT,Edit2,LUID_00003IZ7
0,2023-05-03 13:40:08.387528+00:00,4,00uieujafoYdmkDSx2p7,AS_AT_INSTRUMENT,Original,LUID_00003IZ7
